In [47]:
import numpy as np
import pandas as pd
import requests
import json
import os
import ta

pd.set_option('display.max_columns', 100)


In [51]:
EXCHANGE_ID = "ed14a82e-fbb3-43ed-af33-cb8d99459d39"

BTCUSDT_DAY_ID = "558bb12a-b0a3-463a-a01d-5019fbf353ba"
BTCUSDT_HOUR_ID = "b6aebe76-a1f5-45ec-92a9-e059601473bc"
BTCUSDT_MINUTE_ID = "85d2cfef-ec7d-47cb-8969-439e9c58f352"

ETHUSDT_DAY_ID = "a6f19254-e396-4359-9fd6-b114216ffa06"
ETHUSDT_HOUR_ID = "2c94611b-e12e-4f27-beef-d4e9212f01e7"
ETHUSDT_MINUTE_ID = "4b4488bf-2c76-4e5c-9c71-e6e8f3cccae0"

BNBUSDT_DAY_ID = "e6bd7a29-c2e5-458e-8302-7afc0433f0c6"
BNBUSDT_HOUR_ID = "e4186eef-f39b-4746-9605-5e957c330ca3"
BNBUSDT_MINUTE_ID = "ec51c55e-4e60-4386-9498-b53d06189a30"


ASSET_ID = BTCUSDT_DAY_ID

base_url = "http://172.24.100.128:5000/"
# base_url = "http://localhost:5000/"


PROXIES = {
    
   'http': 'http://discproxy.virtual.uniandes.edu.co:443',
}


In [52]:
# Method to make API request to retrieve prices for a given asset ID between two specified Unix timestamps
def api_request_get_prices_between_unix_time(base_url, asset_id, unix_time_start, unix_time_end):

    # try:
    # Make GET request to API endpoint with query parameters for start and end Unix timestamps
    response = requests.get(base_url + "assets/" + asset_id + "/indicators_unix_between/",
                            params={'unix_time_start': unix_time_start, 'unix_time_end': unix_time_end},proxies=PROXIES)

    # Check if the request was successful
    if response.status_code == requests.codes.ok:
        # print("get prices between status code: "+str(response.status_code))
        # Convert JSON response to Python dictionary
        json_data = json.loads(response.content)
        # Normalize the dictionary and convert it to a Pandas DataFrame
        df = pd.json_normalize(json_data)

        # drop columns that are not needed
        df.drop(columns=['updated_at', 'created_at', 'date_time_gmt_5','ignore','asset_id'], inplace=True)

        # Return the DataFrame containing the retrieved price data
        return df

    else:
        # print("Failed api_request_get_prices_between_unix_time")
        print("get prices between status code: "+str(response.status_code))
        json_data = json.loads(response.content)
        print(json_data)
        return json_data
    


def api_request_get_asset_from_asset_id(base_url, exchange_id, asset_id):
    # try:
    # Make GET request to API endpoint

    response = requests.get(base_url + "exchanges/" +
                            exchange_id + "/asset/"+asset_id,proxies=PROXIES)

    # Check if the request was successful
    if response.status_code == requests.codes.ok:
        print("get asset status code: "+str(response.status_code))
        # Convert JSON response to Python dictionary
        json_data = json.loads(response.content)
        # Normalize the dictionary and convert it to a Pandas DataFrame
        df = pd.json_normalize(json_data)
        # Return the Unix timestamp of the last recorded price for the specified asset ID

        print("symbol: " + df["symbol"].iloc[0])
        print("interval: " + df["interval"].iloc[0])
        return df
    else:
        print(response.content)
        json_data = json.loads(response.content)
        df = pd.json_normalize(json_data)
        return df



# Method to make API request to retrieve prices for a given asset ID between two specified Unix timestamps
def api_request_get_all_indicators_from_price(base_url, price_id,):

    # try:
    # Make GET request to API endpoint with query parameters for start and end Unix timestamps
    response = requests.get(base_url + "prices/" + price_id + "/indicators/",proxies=PROXIES)

    # Check if the request was successful
    if response.status_code == requests.codes.ok:
        # Convert JSON response to Python dictionary
        json_data = json.loads(response.content)
        # Normalize the dictionary and convert it to a Pandas DataFrame
        df = pd.json_normalize(json_data)

        if df.empty:
            return df
        else:
        
            df.drop(columns=['id', 'created_at', 'updated_at','date_time_utc','unix_time','date_time_gmt_5'], inplace=True)

            df.set_index('price_id', inplace=True)

            pivoted_df = pd.pivot_table(df, index='price_id', columns='name', values='value', aggfunc=sum, fill_value=0)

            pivoted_df = pivoted_df.replace(0, np.nan)

            return pivoted_df

    else:
        print("get indicators status code: "+str(response.status_code))
        json_data = json.loads(response.content)
        print(json_data)
        return json_data
    

def get_indicators_and_merge(base_url,exchange_id, asset_id, unix_time_start, unix_time_end):


    # Get the asset from the API
    asset = api_request_get_asset_from_asset_id(base_url,exchange_id,asset_id)

    interval = asset["interval"].iloc[0]
    
    if interval == "minute":
        first_indicator_time = unix_time_start - 6000

    if interval == "hour":
        first_indicator_time = unix_time_start - 360000

    if interval == "day":
        first_indicator_time = unix_time_start - 8640000


    df = api_request_get_prices_between_unix_time(base_url, asset_id, first_indicator_time, unix_time_end)
    
    df_prices_wit_indicators_all = add_indicators(df)

    df_prices_wit_indicators = df_prices_wit_indicators_all.loc[df_prices_wit_indicators_all['unix_time'] >= unix_time_start]

    df_assets = pd.DataFrame()

    for i in df['id']:

        df2 = api_request_get_all_indicators_from_price(base_url, i)

        if not df2.empty:
            df_assets = pd.concat([df_assets, df2])
        
    if not df_assets.empty:
        df_merge = pd.merge(df_prices_wit_indicators, df_assets, left_on='id', right_on='price_id', how='left')
    else:
        df_merge = df_prices_wit_indicators

    return df_merge


# method to add the thecnic indicators to the dataframe
def add_indicators(df):

    # momentum
    df['ao'] = ta.momentum.awesome_oscillator(
        high=df['high_price'], low=df['low_price'])
    df['kama'] = ta.momentum.kama(close=df['close_price'])
    df['ppo'] = ta.momentum.ppo(close=df['close_price'])
    df['pvo'] = ta.momentum.pvo(volume=df['volume'])
    df['roc'] = ta.momentum.roc(close=df['close_price'])
    df['rsi'] = ta.momentum.rsi(close=df['close_price'])
    df['stochrsi'] = ta.momentum.stochrsi(close=df['close_price'])
    df['stoch'] = ta.momentum.stoch(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['tsi'] = ta.momentum.tsi(close=df['close_price'])
    df['uo'] = ta.momentum.ultimate_oscillator(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['wri'] = ta.momentum.williams_r(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])

    # volume
    df['accdist'] = ta.volume.acc_dist_index(
        high=df['high_price'], low=df['low_price'], close=df['close_price'], volume=df['volume'])
    df['cmf'] = ta.volume.chaikin_money_flow(
        high=df['high_price'], low=df['low_price'], close=df['close_price'], volume=df['volume'])
    df['emv'] = ta.volume.ease_of_movement(
        high=df['high_price'], low=df['low_price'], volume=df['volume'])
    df['fi'] = ta.volume.force_index(
        close=df['close_price'], volume=df['volume'])
    df['mfi'] = ta.volume.money_flow_index(
        high=df['high_price'], low=df['low_price'], close=df['close_price'], volume=df['volume'])
    df['nvi'] = ta.volume.negative_volume_index(
        close=df['close_price'], volume=df['volume'])
    df['obv'] = ta.volume.on_balance_volume(
        close=df['close_price'], volume=df['volume'])
    df['smaemv'] = ta.volume.sma_ease_of_movement(
        high=df['high_price'], low=df['low_price'], volume=df['volume'])
    df['vpt'] = ta.volume.volume_price_trend(
        close=df['close_price'], volume=df['volume'])
    df['vwap'] = ta.volume.volume_weighted_average_price(
        high=df['high_price'], low=df['low_price'], close=df['close_price'], volume=df['volume'])

    # volatility
    df['atr'] = ta.volatility.average_true_range(
        high=df['high_price'], low=df['low_price'], close=df['close_price'])
    df['ulcer'] = ta.volatility.ulcer_index(close=df['close_price'])

    df['bbh'] = ta.volatility.bollinger_hband(close=df['close_price'])
    df['bbl'] = ta.volatility.bollinger_lband(close=df['close_price'])
    df['bbhi'] = ta.volatility.bollinger_hband_indicator(
        close=df['close_price'])
    df['bbli'] = ta.volatility.bollinger_lband_indicator(
        close=df['close_price'])
    df['bbmavg'] = ta.volatility.bollinger_mavg(close=df['close_price'])
    df['bb_pb'] = ta.volatility.bollinger_pband(close=df['close_price'])
    df['bb_wb'] = ta.volatility.bollinger_wband(close=df['close_price'])

    df['dchb'] = ta.volatility.donchian_channel_hband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['dclb'] = ta.volatility.donchian_channel_lband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['dcmb'] = ta.volatility.donchian_channel_mband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['dcpb'] = ta.volatility.donchian_channel_pband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['dcwb'] = ta.volatility.donchian_channel_wband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])

    df['kchb'] = ta.volatility.keltner_channel_hband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kclb'] = ta.volatility.keltner_channel_lband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kchbi'] = ta.volatility.keltner_channel_hband_indicator(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kclbi'] = ta.volatility.keltner_channel_lband_indicator(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kcmb'] = ta.volatility.keltner_channel_mband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kcpb'] = ta.volatility.keltner_channel_pband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['kcwb'] = ta.volatility.keltner_channel_wband(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])

    # trend

    # En dos de estos tres hay algo que esta botando warning
    df['adx'] = ta.trend.adx(close=df['close_price'],
                             high=df['high_price'], low=df['low_price'])
    df['adx_neg'] = ta.trend.adx_neg(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['adx_pos'] = ta.trend.adx_pos(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])

    df['aroon_up'] = ta.trend.aroon_up(close=df['close_price'])
    df['aroon_down'] = ta.trend.aroon_down(close=df['close_price'])

    df['cci'] = ta.trend.cci(close=df['close_price'],
                             high=df['high_price'], low=df['low_price'])
    df['dpo'] = ta.trend.dpo(close=df['close_price'])
    df['kst'] = ta.trend.kst(close=df['close_price'])
    df['kst_sig'] = ta.trend.kst_sig(close=df['close_price'])
    df['ema'] = ta.trend.ema_indicator(close=df['close_price'])

    df['ichimoku_a'] = ta.trend.ichimoku_a(
        high=df['high_price'], low=df['low_price'])
    df['ichimoku_b'] = ta.trend.ichimoku_b(
        high=df['high_price'], low=df['low_price'])
    df['ichimoku_base_line'] = ta.trend.ichimoku_base_line(
        high=df['high_price'], low=df['low_price'])
    df['ichimoku_conversion_line'] = ta.trend.ichimoku_conversion_line(
        high=df['high_price'], low=df['low_price'])

    df['macd'] = ta.trend.macd(close=df['close_price'])
    df['macd_diff'] = ta.trend.macd_diff(close=df['close_price'])
    df['macd_signal'] = ta.trend.macd_signal(close=df['close_price'])

    df['mi'] = ta.trend.mass_index(high=df['high_price'], low=df['low_price'])
    df['sma'] = ta.trend.sma_indicator(close=df['close_price'])
    df['wma'] = ta.trend.wma_indicator(close=df['close_price'])
    df['stc'] = ta.trend.stc(close=df['close_price'])
    df['trix'] = ta.trend.trix(close=df['close_price'])

    df['vi_pos'] = ta.trend.vortex_indicator_pos(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])
    df['vi_neg'] = ta.trend.vortex_indicator_neg(
        close=df['close_price'], high=df['high_price'], low=df['low_price'])

    return df

In [53]:
df_prueba = get_indicators_and_merge(base_url, EXCHANGE_ID, ASSET_ID, 1680134400, 1682278312)
df_prueba.sort_values(by='unix_time', ascending=True, inplace=True)
df_prueba.tail(15)

get asset status code: 200
symbol: BTCUSDT
interval: day


/Users/julianoliverosforero/Desktop/2023-1/Tesis/cryptocurrency-market-prediction-project/data-loader/env/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/julianoliverosforero/Desktop/2023-1/Tesis/cryptocurrency-market-prediction-project/data-loader/env/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


,date_time_utc,num_trades,taker_base_vol,open_price,taker_quote_vol,close_price,low_price,id,high_price,volume,unix_time,qav,ao,kama,ppo,pvo,roc,rsi,stochrsi,stoch,tsi,uo,wri,accdist,cmf,emv,fi,mfi,nvi,obv,smaemv,vpt,vwap,atr,ulcer,bbh,bbl,bbhi,bbli,bbmavg,bb_pb,bb_wb,dchb,dclb,dcmb,dcpb,dcwb,kchb,kclb,kchbi,kclbi,kcmb,kcpb,kcwb,adx,adx_neg,adx_pos,aroon_up,aroon_down,cci,dpo,kst,kst_sig,ema,ichimoku_a,ichimoku_b,ichimoku_base_line,ichimoku_conversion_line,macd,macd_diff,macd_signal,mi,sma,wma,stc,trix,vi_pos,vi_neg,NaturalGas,Russell2000,cocoa,coffee,corn,cotton,dowJones,gold,nasdaq,oat,rice,silver,soybean,spy500,sugar,usBond,usBond10y,usBond2y,usBond5y,vwheat
17,2023-04-08T00:00:00+00:00,624159,9729.24214,27906.34,2.723667e+08,27938.38,27859.02,aa9c4e87-5b75-4d4e-9e13-3c6d7de79ddc,28154.99,19479.96735,1680912000,5.453245e+08,1114.384235,27907.660160,2.563191,-44.231843,0.114848,52.588869,0.000000,23.096158,16.382593,44.831520,-76.903842,2.602708e+06,-0.073621,-4.300460e+08,6.324948e+06,37.692231,1456.857143,8.176397e+06,-6.806578e+07,-164.185579,28841.190051,1092.130608,4.984387,30097.827162,26837.855838,0.0,0.0,28467.8415,0.337587,11.451417,30595.60,26631.78,28867.975,0.230962,12.030117,29368.643167,27623.071167,0.0,0.0,28495.857167,0.180634,6.125704,20.418215,22.191536,27.482498,68.0,16.0,-49.371947,-144.0815,148.240749,192.108640,28339.908258,28709.9225,25180.315,28551.870,28867.975,708.252138,-315.839407,1024.091545,24.092073,28811.735833,28275.700667,0.781248,0.597110,0.889040,1.033415,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2023-04-09T00:00:00+00:00,777589,16885.70757,27938.38,4.744543e+08,28323.76,27800.00,0104ad5f-d865-4904-ba59-1fd1528d08ca,28530.00,32531.16101,1680998400,9.139159e+08,2909.492353,26930.675725,4.978148,-45.210872,3.166297,63.502309,0.256495,67.834592,26.499984,51.902577,-32.165408,2.709105e+06,-0.025036,5.195539e+08,5.125751e+07,61.332816,1531.740853,8.440332e+06,8.826497e+07,373.998300,27791.464988,1174.081813,1.613289,31536.297591,21339.201409,0.0,0.0,26437.7495,0.684956,38.570213,29184.68,19765.03,27908.230,0.662768,9.108120,27603.288167,25278.662167,1.0,0.0,26440.975167,1.309930,8.791756,24.798256,18.698201,32.051490,80.0,28.0,57.953147,1025.2005,216.045002,202.554402,27552.303209,26191.5425,22894.980,24474.855,27908.230,1306.552229,1.086778,1305.465451,24.915709,27906.351667,28061.794222,99.995092,0.794008,0.978363,0.866907,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,2023-04-10T00:00:00+00:00,1285695,36147.62326,28323.76,1.043139e+09,29637.34,28170.00,093fb037-dac4-43c4-ad93-d62cc70af961,29770.00,67754.06220,1681084800,1.955301e+09,1222.372529,27921.898382,2.733195,-41.257105,4.637732,61.682115,0.509123,72.266551,17.427549,57.510806,-27.733449,2.659227e+06,-0.040137,2.274095e+09,2.186588e+07,45.509500,1456.857143,8.244151e+06,1.033509e+08,3960.014794,28921.117991,1144.951279,5.002598,30240.262780,26862.349220,0.0,0.0,28551.3060,0.821510,11.831030,30595.60,26631.78,28867.975,0.722666,12.040628,29476.430000,27648.522000,1.0,0.0,28562.476000,1.088030,6.399683,20.909747,19.655750,34.417586,64.0,12.0,58.290240,1649.1140,139.163797,182.526386,28539.513141,28664.4050,25180.315,28551.870,28776.940,759.287845,-211.842960,971.130805,23.954408,28921.200833,28484.755778,0.390624,0.561780,0.901241,0.891092,2.172,1785.599976,2922.0,182.600006,654.00,82.449997,33751.0,1989.099976,13158.00,340.50,1641.0,24.846001,1487.25,4136.25,23.559999,132.56250,115.515625,103.328125,109.937500,876.00
7,2023-04-11T00:00:00+00:00,1340458,34766.92718,29637.35,1.046251e+09,30200.42,29590.00,789e9ecc-43dd-434f-9877-0387b81bf0b9,30550.00,67990.07621,1681171200,2.046078e+09,3108.753176,27143.469213,5.354508,-45.065235,9.967866,70.449471,1.000000,91.351012,28.766459,61.839938,-8.648988,2.727579e+06,-0.011418,2.689804e+09,6.216276e+07,43.453284,1531.740853,8.508322e+06,2.673524e+08,4991.458208,27984.894189,1249.235969,1.613289,31796.821364,21878.923636,